In [1]:
import geopandas as gpd

import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from src.gridify import gdfs_to_grid, point_to_cell, cell_to_polygon, add_courtyards_fast, pois_to_grid_coords, pois_to_grid_coords_adjusted
from src.viz_grid import save_grid_image, grid_to_image,  grid_with_pois_image

from IPython.display import display
import numpy as np


In [2]:
buildings_gdf = gpd.read_file("../data/2024-geometries/2024_Edifici_EPSG32633.geojson")
streets_gdf = gpd.read_file("../data/2024-geometries/2024_Streets_EPSG32633.geojson")
canals_gdf = gpd.read_file("../data/2024-geometries/2024_Canals_EPSG32633.geojson")

In [3]:
buildings_gdf.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
grid, tr, legend = gdfs_to_grid(buildings_gdf, streets_gdf, canals_gdf, cell_size=1)
print(legend)          # {0:'ocean', 1:'street', 2:'building', 3:'canal'}


{0: 'ocean', 1: 'street', 2: 'building', 3: 'canal'}


In [5]:
grid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(6841, 4908), dtype=uint8)

In [6]:
tr

Affine(1.0, 0.0, 289084.0,
       0.0, -1.0, 5037646.0)

In [7]:
grid, legend = add_courtyards_fast(grid)

In [8]:
type(grid)

numpy.ndarray

In [9]:
# np.save(grid, )

In [10]:
# 1× zoom (one pixel per cell) – smallest file size
save_grid_image(grid, "city_1x.png", scale=1)
# save_grid_image(grid, "city_2x.png", scale=2)

# 5× zoom – clearer when you open it in an image viewer
# img = grid_to_image(grid, scale=5)

# Adding data from the previous centuries

In [11]:
catastaci_1740 = gpd.read_file('../data/1740-catastici/1740_catastici_version20250625.geojson')

In [12]:
catastaci_1740.columns

Index(['uid', 'author', 'tif_path_img', 'owner_code', 'owner_count',
       'PP_OwnerCode_SIMPL', 'owner_name', 'ten_name', 'function', 'an_rendi',
       'id_napo', 'quantity_income', 'quality_income', 'place', 'parish_std',
       'sestiere', 'PP_Function_TOP', 'PP_Function_MID',
       'PP_Function_PROPERTY', 'PP_Function_GEOMETRY', 'PP_Bottega_STD',
       'PP_Bottega_COUNT', 'PP_Bottega_TRAD', 'PP_Bottega_METACATEGORY',
       'PP_Owner_Title', 'PP_Owner_Entity', 'PP_Owner_FirstName',
       'PP_Owner_LastName', 'owner_mestiere_std', 'PP_Owner_Notes',
       'geometry'],
      dtype='object')

In [13]:
catastaci_1740.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
catastaci_1740.PP_Function_TOP.value_counts()

PP_Function_TOP
CASA          22592
MIXED          3572
BOTTEGA        3339
MAGAZZINO       993
TRAGHETTO       490
INVIAMENTO      434
OTHER           417
Name: count, dtype: int64

In [15]:
catastaci_1740.PP_Bottega_METACATEGORY.value_counts()

PP_Bottega_METACATEGORY
FOOD_DRINK             955
COMMERCE_CLOTHING      374
HEALTH_BEAUTY          352
WOOD_CRAFT             181
LUXURY_ITEMS           168
METAL_CRAFT            151
FOOD_RAW               114
HOUSEHOLD_ITEMS        111
FABRIC_SALE            110
ARTS_CRAFT              81
MIXED                   80
BUILDING                74
FABRIC_CRAFT            58
SERVICES                45
OTHER                   28
STONE_CRAFT             25
PAPER_SALE              24
ARTS_CRAFT_SUPPLIES     19
TRANSPORT               14
LIBERAL_PROFESSION      12
WOOD_SALE               11
PAPER_CRAFT              7
BOOK_SALE                6
WEAPONS                  6
GLASS_CRAFT              2
GLASS_SALE               1
Name: count, dtype: int64

# POIs to Grid Cells

In [16]:
poi_gdf = catastaci_1740
poi_gdf = pois_to_grid_coords(poi_gdf, tr, grid.shape)        # adds 'row', 'col'

In [17]:
poi_gdf

,uid,author,tif_path_img,owner_code,owner_count,PP_OwnerCode_SIMPL,owner_name,ten_name,function,an_rendi,...,PP_Bottega_METACATEGORY,PP_Owner_Title,PP_Owner_Entity,PP_Owner_FirstName,PP_Owner_LastName,owner_mestiere_std,PP_Owner_Notes,geometry,row,col
0,CNC-0001,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Liberal Campi secondo prete titolato della Chiesa,Francesco Zeni,casa e bottega da barbier,70,...,HEALTH_BEAUTY,SECONDO PRETE,None,Liberal,CAMPI,,PERSON,POINT (291832.846 5035321.328),2324,2748
1,CNC-0002,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Filippo Frari terzo prete titolato della Chiesa,Dio M'aiuti Lazara,casa,60,...,None,TERZO PRETE,None,Filippo,FRARI,,PERSON,POINT (291841.531 5035307.749),2338,2757
2,CNC-0003,Davide Drago,/catastici/Catastici-436/10/1393.tif,ent_REL_TTL,1,Religious entity,Pievan di San Cancian,Bortolamio Piazza,bottega da strazariol,14,...,COMMERCE_CLOTHING,PIEVANO,CHIESA DI SAN CANCIANO,None,None,,ENTITY,POINT (291845.478 5035313.512),2332,2761
3,CNC-0004,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Filippo Frari terzo prete titolato della Chiesa,Bortolamio Piazza,bottega da strazariol,4,...,COMMERCE_CLOTHING,TERZO PRETE,None,Filippo,FRARI,,PERSON,POINT (291846.426 5035312.091),2333,2762
4,CNC-0005,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Agostin Filippi di Vicenza,Stefano Ratti,casa e bottega da tentor,70,...,FABRIC_CRAFT,,None,Agostin,FILIPPI,,PERSON,POINT (291824.083 5035303.564),2342,2740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32671,APO-0415,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Girolamo Giacomini,nan,abitazione di propria proprietà,nan,...,None,,None,Girolamo,GIACOMINI,,PERSON,POINT (291255.085 5035063.179),2582,2171
32672,APO-0416,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Giannantonio Bonicelli,nan,abitazione di propria proprietà,nan,...,None,,None,Giannantonio,BONICELLI,,PERSON,POINT (291243.71 5035150.776),2495,2159
32673,APO-0417,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Ferdinando Crivelli,nan,abitazione di propria proprietà,nan,...,None,,None,Ferdinando,CRIVELLI,,PERSON,POINT (291287.848 5035146.617),2499,2203
32674,APO-0418,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Bortolo Bernardi,nan,abitazione di propria proprietà,nan,...,None,,None,Bortolo,BERNARDI,,PERSON,POINT (291280.549 5035138.808),2507,2196


In [18]:
poi_gdf.columns

Index(['uid', 'author', 'tif_path_img', 'owner_code', 'owner_count',
       'PP_OwnerCode_SIMPL', 'owner_name', 'ten_name', 'function', 'an_rendi',
       'id_napo', 'quantity_income', 'quality_income', 'place', 'parish_std',
       'sestiere', 'PP_Function_TOP', 'PP_Function_MID',
       'PP_Function_PROPERTY', 'PP_Function_GEOMETRY', 'PP_Bottega_STD',
       'PP_Bottega_COUNT', 'PP_Bottega_TRAD', 'PP_Bottega_METACATEGORY',
       'PP_Owner_Title', 'PP_Owner_Entity', 'PP_Owner_FirstName',
       'PP_Owner_LastName', 'owner_mestiere_std', 'PP_Owner_Notes', 'geometry',
       'row', 'col'],
      dtype='object')

In [19]:
poi_gdf.ten_name.value_counts()

ten_name
nan                 2981
Antonio Rossi         22
Antonio Calvi         18
Francesco Rossi       17
Bortolo Usabelli      16
                    ... 
Iseppo Biffon          1
Domenico Garbato       1
Pietro Violi           1
Zuanne Zulian          1
Maria Capellani        1
Name: count, Length: 23985, dtype: int64

In [20]:
mixed_pois = poi_gdf[poi_gdf['PP_Function_TOP'] == "MIXED"]

In [21]:
mixed_pois.PP_OwnerCode_SIMPL.value_counts()

PP_OwnerCode_SIMPL
Private                  2997
Religious entity          334
Scuola                    110
Other                      54
Social institution         36
Unknown                    32
Repubblica di Venezia       9
Name: count, dtype: int64

In [22]:
color_map = {
    "CASA":        (213,  94,   0),   # burnt orange
    "MIXED":       (  0, 158, 115),   # teal green
    "BOTTEGA":     (230, 159,   0),   # golden yellow
    "MAGAZZINO":   ( 86, 180, 233),   # sky blue
    "TRAGHETTO":   (  0, 114, 178),   # deep blue
    "INVIAMENTO":  (240, 228,  66),   # bright yellow
    "OTHER":       (204, 121, 167),   # mauve pink
}

In [23]:
img = grid_with_pois_image(
    grid, tr, poi_gdf,
    scale=5,
    color_map=color_map,      # or leave None for auto colors
)
img.save("venice_2024_pois_by_function.png")

# Sommarioni Example

In [24]:
sommarioni_gdf = gpd.read_file('../data/1808-sommarioni/venice_1808_landregister_geometries.geojson')

In [25]:
sommarioni_gdf.columns

Index(['id', 'parcel_number', 'geometry_type', 'geometry_id',
       'parish_standardised', 'area', 'geometry'],
      dtype='object')

In [26]:
smg_tr_3857 = sommarioni_gdf.to_crs("EPSG:3857")

In [27]:
# sommarioni_layers.py
"""
End-to-end utilities for 1808 Venice land‐register data.

Dependencies
------------
geopandas, rasterio, shapely, numpy
plus the earlier grid helpers:
    • gdfs_to_grid (gridify.py)
    • add_courtyards_fast (optional, gridify.py)

State codes used in the resulting grid
--------------------------------------
0 ocean/background
1 street
2 building (incl. sottoportico)
3 canal / lagoon
4 courtyard (manual polygons first, algorithmic fill-ins optional)
"""

from __future__ import annotations
import geopandas as gpd
import numpy as np
from affine import Affine
from rasterio import features


def load_venice_layers(
    json_path: str,
    target_crs: int | str = 32633,
) -> tuple[gpd.GeoDataFrame, gpd.GeoDataFrame,
           gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Read the 1808 GeoJSON and return four GeoDataFrames:
        buildings_gdf, streets_gdf, canals_gdf, courtyards_gdf
    All re-projected to *target_crs* (EPSG:3857 metres by default).
    """
    gdf = gpd.read_file(json_path)
    if target_crs:
        gdf = gdf.to_crs(target_crs)

    buildings = gdf[gdf["geometry_type"] == 'building'].copy()
    streets   = gdf[gdf["geometry_type"].isin(["street", "sottoportico"])].copy()
    canals    = gdf[gdf["geometry_type"] == "water"].copy()
    courtyards = gdf[gdf["geometry_type"] == "courtyard"].copy()

    # occasionally sottoportici are MultiPolygons: explode for cleaner raster
    buildings = buildings.explode(index_parts=False)

    return buildings, streets, canals, courtyards


def venice_grid(
    json_path: str,
    cell_size: float = 1.0,
    target_crs: int | str = 32633,
    *,
    include_auto_courtyards: bool = True,
):
    """
    Build a uint8 grid from the 1808 land-register GeoJSON.

    Returns
    -------
    grid : np.ndarray  (rows, cols)
    transform : affine.Affine
    legend : dict[int, str]
    layers : dict[str, gpd.GeoDataFrame]
        {"buildings":…, "streets":…, "canals":…, "courtyards":…}
    """
    buildings, streets, canals, courtyards = load_venice_layers(
        json_path, target_crs
    )

    grid, transform, legend = gdfs_to_grid(
        buildings, streets, canals=canals, cell_size=cell_size
    )

    # --- manual courtyards (parcel_type == 'courtyard') ---------------------
    _burn_polygons_to_grid(
        courtyards.geometry, grid, transform, value=4
    )
    legend[4] = "courtyard"

    # --- optional algorithmic flood-fill of *remaining* interior empties ----
    if include_auto_courtyards:
        add_courtyards_fast(grid)        # only touches empty pockets

    layers = {
        "buildings": buildings,
        "streets": streets,
        "canals": canals,
        "courtyards": courtyards,
    }
    return grid, transform, legend, layers


# ----- internal utility -----------------------------------------------------

def _burn_polygons_to_grid(
    geoms,
    grid: np.ndarray,
    transform: Affine,
    *,
    value: int,
):
    """
    Rasterise a set of polygons INTO an existing uint8 grid (in-place).
    """
    shapes = ((geom, value) for geom in geoms if not geom.is_empty)
    features.rasterize(
        shapes=shapes,
        out=grid,
        transform=transform,
        all_touched=True,
        default_value=value,
    )


In [28]:
GRID_JSON = "../data/1808-sommarioni/venice_1808_landregister_geometries.geojson"

grid, tr, legend, layers = venice_grid(
    GRID_JSON,
    cell_size=1,              # 1-metre resolution
    include_auto_courtyards=True,
)

print(legend)
# {'ocean':0, 'street':1, 'building':2, 'canal':3, 'courtyard':4}
save_grid_image(grid, "city_sommarioni_1x.png", scale=1)

# optional: overlay POIs, color-map defined earlier
# img = grid_with_pois_image(grid, tr, poi_gdf, scale=5, color_map=color_map)
# img.save("venice_full_5x.png")

{0: 'ocean', 1: 'street', 2: 'building', 3: 'canal', 4: 'courtyard'}


In [29]:
grid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(3180, 4994), dtype=uint8)

In [30]:
catastaci_1740.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
tr

Affine(1.0, 0.0, 289316.0,
       0.0, -1.0, 5036666.0)

In [32]:
grid, tr, legend, layers = venice_grid(GRID_JSON,
                                       cell_size=1, target_crs=32633)

poi_gdf = pois_to_grid_coords(catastaci_1740.to_crs(32633), tr, grid.shape)  # now keeps its rows
print(len(poi_gdf), "POIs landed inside the grid")


32676 POIs landed inside the grid


In [33]:
poi_gdf.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [34]:
# optional: overlay POIs, color-map defined earlier
img = grid_with_pois_image(grid, tr, poi_gdf, scale=5, color_map=color_map)
img.save("venice_sommarioni_5x.png")

KeyboardInterrupt: 

### POIs transformed/adjusted

In [36]:
grid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(3180, 4994), dtype=uint8)

In [38]:
poi_gdf = pois_to_grid_coords_adjusted(catastaci_1740.to_crs(32633), tr, grid)  # now keeps its rows
print(len(poi_gdf), "POIs landed inside the grid")

32676 POIs landed inside the grid


In [41]:
poi_gdf_adj = poi_gdf.rename(columns={
    'row': 'row_orig',
    'col': 'col_orig',
    'row_adj': 'row',
    'col_adj': 'col'
})
poi_gdf_adj

,uid,author,tif_path_img,owner_code,owner_count,PP_OwnerCode_SIMPL,owner_name,ten_name,function,an_rendi,...,PP_Owner_Entity,PP_Owner_FirstName,PP_Owner_LastName,owner_mestiere_std,PP_Owner_Notes,geometry,row_orig,col_orig,row,col
0,CNC-0001,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Liberal Campi secondo prete titolato della Chiesa,Francesco Zeni,casa e bottega da barbier,70,...,None,Liberal,CAMPI,,PERSON,POINT (291832.846 5035321.328),1344,2516,1344,2515
1,CNC-0002,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Filippo Frari terzo prete titolato della Chiesa,Dio M'aiuti Lazara,casa,60,...,None,Filippo,FRARI,,PERSON,POINT (291841.531 5035307.749),1358,2525,1359,2523
2,CNC-0003,Davide Drago,/catastici/Catastici-436/10/1393.tif,ent_REL_TTL,1,Religious entity,Pievan di San Cancian,Bortolamio Piazza,bottega da strazariol,14,...,CHIESA DI SAN CANCIANO,None,None,,ENTITY,POINT (291845.478 5035313.512),1352,2529,1352,2521
3,CNC-0004,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Filippo Frari terzo prete titolato della Chiesa,Bortolamio Piazza,bottega da strazariol,4,...,None,Filippo,FRARI,,PERSON,POINT (291846.426 5035312.091),1353,2530,1353,2521
4,CNC-0005,Davide Drago,/catastici/Catastici-436/10/1393.tif,PPL,1,Private,Agostin Filippi di Vicenza,Stefano Ratti,casa e bottega da tentor,70,...,None,Agostin,FILIPPI,,PERSON,POINT (291824.083 5035303.564),1362,2508,1361,2509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32671,APO-0415,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Girolamo Giacomini,nan,abitazione di propria proprietà,nan,...,None,Girolamo,GIACOMINI,,PERSON,POINT (291255.085 5035063.179),1602,1939,1603,1939
32672,APO-0416,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Giannantonio Bonicelli,nan,abitazione di propria proprietà,nan,...,None,Giannantonio,BONICELLI,,PERSON,POINT (291243.71 5035150.776),1515,1927,1516,1925
32673,APO-0417,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Ferdinando Crivelli,nan,abitazione di propria proprietà,nan,...,None,Ferdinando,CRIVELLI,,PERSON,POINT (291287.848 5035146.617),1519,1971,1523,1967
32674,APO-0418,Davide Drago,/catastici/Catastici-437/6/0736.tif,PPL,1,Private,Bortolo Bernardi,nan,abitazione di propria proprietà,nan,...,None,Bortolo,BERNARDI,,PERSON,POINT (291280.549 5035138.808),1527,1964,1527,1959


In [43]:
# optional: overlay POIs, color-map defined earlier
img = grid_with_pois_image(grid, tr, poi_gdf_adj, scale=5, color_map=color_map)
img.save("venice_sommarioni_5x_adj.png")